In [12]:
from __future__ import division, print_function
import os
import ipyparallel as ipp
rc = ipp.Client()
v = rc.load_balanced_view()

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

signal_opts={'signal':3.5, 
             'n':3000, 
             'p':1000, 
             'k':30,
             'sigma':2}



In [13]:
%%px

signal_opts={'signal':3.5, 
             'n':3000, 
             'p':1000, 
             'k':30,
             'sigma':2}

from __future__ import division, print_function
from IPython.display import HTML

%load_ext rpy2.ipython
import rpy2.robjects as rpy
import numpy as np
import statsmodels.api as sm

# this code below is available at https://github.com/jonathan-taylor/selective-inference
from selection.algorithms.lasso import lasso
from selection.algorithms.sqrt_lasso import choose_lambda

%R library(glmnet)
%R library(selectiveInference)
%R library(knockoff)

[stdout:0] 
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
[stdout:1] 
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
[stdout:2] 
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Out[0:6]: 
array(['knockoff', 'selectiveInference', 'survival', 'intervals', 'glmnet',
       'foreach', 'Matrix', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='|S18')

Out[1:6]: 
array(['knockoff', 'selectiveInference', 'survival', 'intervals', 'glmnet',
       'foreach', 'Matrix', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='|S18')

Out[2:6]: 
array(['knockoff', 'selectiveInference', 'survival', 'intervals', 'glmnet',
       'foreach', 'Matrix', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='|S18')

## Data generating mechanism

We use the same data generating mechanism as in [Barber and Candes (2016)](http://arxiv.org/abs/1602.03574).

In [14]:
%%px
def cov(p, rho=0.25):
    idx = np.arange(p)
    return rho**np.fabs(np.subtract.outer(idx, idx))

def sqrt_cov(p, rho=0.25):
    idx = np.arange(p)
    C = rho**np.fabs(np.subtract.outer(idx, idx))
    return np.linalg.cholesky(C)

# Testing we have the square-root correct
p = 2500
A = cov(p, rho=0.3)
B = sqrt_cov(p, rho=0.3)
np.linalg.norm(B.dot(B.T) - A)

Out[0:7]: 3.6286017038963421e-15

Out[1:7]: 3.6286017038963421e-15

Out[2:7]: 3.6286017038963421e-15

In [15]:
%%px
cholesky_factors = {}
for rho in [0, 0.25, 0.5, 0.75]:
    cholesky_factors[(rho, 2500)] = sqrt_cov(2500, rho=rho)

In [16]:
%%px
def instance(n=1000, k=30, p=400, signal=3.5, rho=0.25, sigma=1.234): # sigma there just to convince you 
                                                                       # we don't need to know noise level
    if (rho, p) in cholesky_factors.keys():
        _sqrt_cov = cholesky_factors[(rho, p)]
    else:
        cholesky_factors[(rho, p)] = sqrt_cov(p, rho=rho)
        _sqrt_cov = cholesky_factors[(rho, p)]
    X = np.random.standard_normal((n, p)).dot(_sqrt_cov.T)

    X /= (np.sqrt((X**2).sum(0))) # like normc
    beta = np.zeros(p)
    beta[:k] = signal * (2 * np.random.binomial(1, 0.5, size=(k,)) - 1) 
    np.random.shuffle(beta)

    Y = (X.dot(beta) + np.random.standard_normal(n)) * sigma
    true_active = np.nonzero(beta != 0)[0]
    return X, Y, true_active, beta

X, Y, true_active, beta = instance()


In [17]:
%%px
def simulate(rho=np.arange(4)/4., q=0.2, kappa=[0.7], do_cv=True, do_knockoff=True, signal_opts=signal_opts):
    
    n = signal_opts['n']
    p = signal_opts['p']
    sigma = signal_opts['sigma']
    P0, PA, active_size = {}, {}, {}
    full_model_FDP, full_model_power, directional_FDP = {}, {}, {}
    effective_lam = {}
    kappa = sorted(kappa)[::-1]

    for r in rho:
        signal_opts['rho'] = r
        X, Y, true_active, beta = instance(**signal_opts)

        theory_lam_lasso = np.mean(np.fabs(np.dot(X.T, np.random.standard_normal((n, 2000)))).max(0) * sigma)

        # square-root LASSO at several kappa values

        for _kap in kappa:

            lam = choose_lambda(X)
            L = lasso.sqrt_lasso(X, Y, _kap * lam)
            L.fit()
            S = L.summary('onesided')
            
            _kap = r'$\kappa=%0.2f$' % _kap
            active_size.setdefault((_kap, r), []).append(len(L.active))
            effective_lam.setdefault((_kap, r), []).append(L._penalty.weights[0] / theory_lam_lasso)
            
            # keep p-values when screening 
        
            if set(L.active).issuperset(true_active):
                p0 = [_pv for _pv, v in zip(S['pval'], S['variable']) if v not in true_active]
                pA = [_pv for _pv, v in zip(S['pval'], S['variable']) if v in true_active]
            else:
                p0 = []
                pA = []
                
            P0.setdefault((_kap, r), []).extend(p0) 
            PA.setdefault((_kap, r), []).extend(pA)
            if len(L.active) > 0:
                selected = sm.stats.multipletests(S['pval'], q, 'fdr_bh')[0]
            else:
                selected = np.zeros(S.shape, np.bool)
            type1_errors = selected * np.array([v not in true_active for v in S['variable']])
            if hasattr(L, 'active_signs'):
                sign_errors = selected * np.array([(s != np.sign(beta[v])) * (v in true_active) for v, s in 
                                                   zip(S['variable'], L.active_signs)])
            else:
                sign_errors = 0

            full_model_FDP.setdefault((_kap, r), []).append(np.sum(type1_errors) / max(np.sum(selected), 1))
            directional_FDP.setdefault((_kap, r), []).append(np.sum(type1_errors + sign_errors) / max(np.sum(selected), 1))
            full_model_power.setdefault((_kap, r), []).append(np.sum(selected * np.array([v in true_active for v in S['variable']])) / len(true_active))
                
        # now do cv.glmnet and Lee et al. with
        
        if do_cv:
            
            %R -i X,Y Y = as.matrix(Y)
            %R X = as.matrix(X)
            %R G_CV = cv.glmnet(X, Y, standardize=FALSE, intercept=FALSE)
            %R sigma_hat = estimateSigma(X, Y, standardize=FALSE, intercept=FALSE)$sigmahat
            %R lamhat = G_CV$lambda.min
            %R fit = glmnet(X, Y, standardize=FALSE,intercept=FALSE)
            %R Yhat = predict(fit, X, s = lamhat)
            %R nz = sum(predict(fit, s = lamhat, type = "coef") != 0)
            %R sigma_hat = sqrt(sum((Y - Yhat)^2)/(length(Y) - nz - 1))

            sigma_hatR = %R sigma_hat
            lam_1SE = %R G_CV$lambda.1se
            lam_minCV = %R G_CV$lambda.min
    
            lam_1SE *= n
            lam_minCV *= n

            for method, lam, sigma_hat in zip(['1SE', 'Lee et al.'], 
                                              [float(lam_1SE), theory_lam_lasso],
                                              [float(sigma_hatR), sigma]):
                L = lasso.gaussian(X, Y, float(lam), sigma=sigma_hat)           
                L.fit() 
                S = L.summary('onesided')

                active_size.setdefault((method, r), []).append(len(L.active))
                effective_lam.setdefault((method, r), []).append(L._penalty.weights[0] / theory_lam_lasso * sigma**2)

                # keep p-values when screening 
        
                if set(L.active).issuperset(true_active):
                    p0 = [_pv for _pv, v in zip(S['pval'], S['variable']) if v not in true_active]
                    pA = [_pv for _pv, v in zip(S['pval'], S['variable']) if v in true_active]
                else:
                    p0 = []
                    pA = []

                P0.setdefault((method, r), []).extend(p0) 
                PA.setdefault((method, r), []).extend(pA)

                if len(L.active) > 0:
                    selected = sm.stats.multipletests(S['pval'], q, 'fdr_bh')[0]
                else:
                    selected = np.zeros(S.shape, np.bool)

                type1_errors = selected * np.array([v not in true_active for v in S['variable']])
                if hasattr(L, 'active_signs'):
                    sign_errors = selected * np.array([(s != np.sign(beta[v])) * (v in true_active) for v, s in 
                                                       zip(S['variable'], L.active_signs)])
                else:
                    sign_errors = 0

                full_model_FDP.setdefault((method, r), []).append(np.sum(type1_errors) / max(np.sum(selected), 1))
                directional_FDP.setdefault((method, r), []).append(np.sum(type1_errors + sign_errors) / max(np.sum(selected), 1))
                full_model_power.setdefault((method, r), []).append(np.sum(selected * np.array([v in true_active for v in S['variable']])) / len(true_active))

        if (signal_opts['n'] > 2*signal_opts['p']) and do_knockoff:

            method = 'knockoff'
            %R -i q
            %R Y=as.matrix(Y)
            %R Y=as.numeric(Y)
            %R X=as.matrix(X)
            %R KO=knockoff.filter(X=X, y=Y, fdr=q, threshold="knockoff")
            selected_idx = rpy.r('KO$selected')
            selected = np.zeros(p, np.bool)
            if selected_idx is not None:
                selected_idx = np.array(selected_idx) - 1 # R vs. python indexing
                selected[selected_idx] = True
                            
            type1_errors = selected * np.array([v not in true_active for v in np.arange(p)])
            sign_errors = 0 # don't know how to extract signs from R package

            full_model_FDP.setdefault((method, r), []).append(np.sum(type1_errors) / max(np.sum(selected), 1))
            directional_FDP.setdefault((method, r), []).append(np.sum(type1_errors + sign_errors) / max(np.sum(selected), 1))
            full_model_power.setdefault((method, r), []).append(np.sum(selected * np.array([v in true_active for v in np.arange(p)])) / len(true_active))
            
    return P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam

In [18]:
@v.parallel(block=False)
def batch(nullarg, **signal_opts):
    return simulate(signal_opts=signal_opts)

In [19]:
def save_data(outfilename, P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam):
    
    if os.path.exists(outfilename):
        old_results = np.load(outfilename)
    else:
        old_results = None

    results = {}
    
    for v, n in zip([P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam],
                    ['P0', 'PA', 'full_model_FDP', 'directional_FDP', 'full_model_power', 'active_size', 'effective_lam']):
        for key in v.keys():
            nkey = "__".join([n] + [str(k) for k in key])
            if old_results == None:
                results[nkey] = v[key]
            elif nkey in old_results:
                results[nkey] = np.hstack([old_results[nkey], v[key]])
    np.savez(outfilename, **results)
    
def load_data(infilename):
    
    results = np.load(infilename)

    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = {}, {}, {}, {}, {}, {}, {}
    for v, n in zip([P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam],
                    ['P0', 'PA', 'full_model_FDP', 'directional_FDP', 'full_model_power', 'active_size', 'effective_lam']):
        keys = [(key, key.split('__')[1:]) for key in results.keys() if key[:len(n)] == n]
        for k1, k2 in keys:
            _k, r = k2
            r = float(r)
            v[(_k,r)] = results[k1]

    return P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam

In [20]:
def figure_panel(filename):
    
    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = load_data(filename)

    first_fig = plt.figure(figsize=(20,7))

    # Model FDR

    modelFDP_ax = plt.subplot(1,3,1)
    modelFDP_dict = {}
    for m, r in sorted(full_model_FDP.keys()):
        modelFDP_dict.setdefault(m, []).append((r, np.mean(full_model_FDP[(m, r)])))
    for m in sorted(modelFDP_dict.keys()):
        modelFDP_dict[m] = np.array(modelFDP_dict[m])
        modelFDP_ax.plot(modelFDP_dict[m][:,0], modelFDP_dict[m][:,1], label=m)
    modelFDP_ax.legend(loc='lower left')
    modelFDP_ax.set_ylabel(r'E(Model FDP)($\rho$)', fontsize=20)
    modelFDP_ax.set_xlabel(r'$\rho$', fontsize=20)

    # Directional FDR

    dirFDP_ax =  plt.subplot(1,3,2)
    dirFDP_dict = {}
    for m, r in sorted(full_model_FDP.keys()):
        dirFDP_dict.setdefault(m, []).append((r, np.mean(directional_FDP[(m, r)])))
    for m in sorted(modelFDP_dict.keys()):
        dirFDP_dict[m] = np.array(modelFDP_dict[m])
        dirFDP_ax.plot(dirFDP_dict[m][:,0], dirFDP_dict[m][:,1], label=m)
    dirFDP_ax.legend(loc='lower left')
    dirFDP_ax.set_ylabel(r'E(Directional FDP)($\rho$)', fontsize=20)
    dirFDP_ax.set_xlabel(r'$\rho$', fontsize=20)

    # Power 

    power_ax = plt.subplot(1,3,3)
    power_dict = {}
    for m, r in sorted(full_model_FDP.keys()):
        power_dict.setdefault(m, []).append((r, np.mean(full_model_power[(m, r)])))
    for m in sorted(power_dict.keys()):
        power_dict[m] = np.array(power_dict[m])
        power_ax.plot(power_dict[m][:,0], power_dict[m][:,1], label=m)
    power_ax.legend(loc='lower left')
    power_ax.set_ylabel(r'Power($\rho$)', fontsize=20)
    power_ax.set_xlabel(r'$\rho$', fontsize=20)
    
    first_fig.savefig(filename.split('.')[0] + '_figure_panel.pdf')
    plt.close(first_fig)
    
def figure_null(filename):
    
    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = load_data(filename)

    null_fig = plt.figure(figsize=(12,12))
    U = np.linspace(0, 1, 201)
    null_ax =  null_fig.gca()
    null_dict = {}
    for m, r in sorted(P0.keys()):
        null_dict.setdefault(m, []).extend(P0[(m,r)])
    for m in sorted(null_dict.keys()):
        if null_dict[m]:
            null_ax.plot(U, sm.distributions.ECDF(null_dict[m])(U), label=m)
    null_ax.legend(loc='lower right')
    null_ax.set_ylabel(r'ECDF(p)', fontsize=20)
    null_ax.set_xlabel(r'$p', fontsize=20)

    null_fig.savefig(filename.split('.')[0] + '_figure_null.pdf')
    plt.close(null_fig)
    
def figure_alt(filename):
    
    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = load_data(filename)

    alt_fig = plt.figure(figsize=(12,12))
    U = np.linspace(0, 1, 201)
    alt_ax = alt_fig.gca()
    alt_dict = {}
    for m, r in sorted(PA.keys()):
        alt_dict.setdefault(m, []).extend(PA[(m,r)])
    for m in sorted(alt_dict.keys()):
        if alt_dict[m]:
            alt_ax.plot(U, sm.distributions.ECDF(alt_dict[m])(U), label=m)
    alt_ax.legend(loc='lower right')
    alt_ax.set_ylabel(r'ECDF(p)', fontsize=20)
    alt_ax.set_xlabel(r'$p', fontsize=20)

    alt_fig.savefig(filename.split('.')[0] + '_figure_alt.pdf')
    plt.close(alt_fig)

def figure_active_set(filename):
    
    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = load_data(filename)

    active_fig = plt.figure(figsize=(12,12))
    active_ax = active_fig.gca()
    active_dict = {}
    for m, r in sorted(active_size.keys()):
        active_dict.setdefault(m, []).append((r, np.mean(active_size[(m, r)])))
    for m in sorted(active_dict.keys()):
        active_dict[m] = np.array(active_dict[m])
        active_ax.plot(active_dict[m][:,0], active_dict[m][:,1], label=m)
    active_ax.legend(loc='lower left')
    active_ax.set_ylabel(r'E(# active)($\rho$)', fontsize=20)
    active_ax.set_xlabel(r'$\rho$', fontsize=20)

    active_fig.savefig(filename.split('.')[0] + '_figure_active.pdf')
    plt.close(active_fig)
    
def figure_lam(filename):
    
    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = load_data(filename)

    lam_fig = plt.figure(figsize=(12,12))
    lam_ax = lam_fig.gca()
    lam_dict = {}
    for m, r in sorted(effective_lam.keys()):
        lam_dict.setdefault(m, []).append((r, np.mean(effective_lam[(m, r)])))
    for m in sorted(lam_dict.keys()):
        lam_dict[m] = np.array(lam_dict[m])
        lam_ax.plot(lam_dict[m][:,0], lam_dict[m][:,1], label=m)
    lam_ax.legend(loc='lower left')
    lam_ax.set_ylabel(r'E($\hat{\lambda} / \lambda_{theory}$)($\rho$)', fontsize=20)
    lam_ax.set_xlabel(r'$\rho$', fontsize=20)

    lam_fig.savefig(filename.split('.')[0] + '_figure_lam.pdf')
    plt.close(lam_fig)



In [ ]:
def make_plots(nsim=100, signal_opts=signal_opts):
    
    @v.parallel(block=False)
    def batch(nullarg):
        return simulate(signal_opts=signal_opts)
    results = batch.map(range(nsim))

    P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam = {}, {}, {}, {}, {}, {}, {}
    for i, r in enumerate(results):
        _P0, _PA, _full_model_FDP, _directional_FDP, _full_model_power, _active_size, _effective_lam = r
        for d1, d2 in zip([P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam],
                         [_P0, _PA, _full_model_FDP, _directional_FDP, _full_model_power, _active_size, _effective_lam]):
            for key in d2.keys():
                d1.setdefault(key,[]).extend(d2[key])
        save_data('lowdim.npz', P0, PA, full_model_FDP, directional_FDP, full_model_power, active_size, effective_lam)
        print('result %d received' % i)
        if True:
        
            figure_panel('lowdim.npz')
            figure_null('lowdim.npz')
            figure_alt('lowdim.npz')
            figure_lam('lowdim.npz')
            figure_active_set('lowdim.npz')

In [ ]:
make_plots()